In [1]:
# imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import classification_report
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
from sklearn.svm import SVC
from sklearn import svm
from sklearn import grid_search
import random

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

/Users/Cyril/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Cyril/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


<b> Defining the filling empty values function </b>

In [17]:
import sys

def fillEmpty(original_df,colnum,flag):
    # assumed: df is the dataframe to operate on,
    # colnum is the column number with missing values
    # flag = 0/1 is whether it is a classification or regression problem
    
    # copying the original dataframe
    df = original_df.copy()
    
    # testing for valid flag
    if(flag != 0 and flag != 1):
        print('Invalid input flag')
        sys.exit()
    # testing for valid column number
    number_of_columns = len(list(original_df))
    if(colnum < 0 or colnum >= number_of_columns):
        print('Invalid input column number')
        sys.exit()
    # testing for the existence of empty column values
    a = df.iloc[:, [colnum]].isnull()
    idx = []
    for col in a:
        i=0
        for c in a[col]:
            if(c == True):
                idx.append(i)
            i=i+1
    if(len(idx) == 0):
        print('No empty values for input column number')
        sys.exit()
    
    # now can start pre-processing:
    # This converts all columns with "object" variables (AKA string) into numbers, and creates a dictionary  
    char_cols = df.dtypes.pipe(lambda x: x[x == 'object']).index
    label_mapping = {}
    for c in char_cols:
        df[c], label_mapping[c] = pd.factorize(df[c])
    # although some issue with the above may arise if null:
    # re-set all null values to null
    df.iloc[idx,[colnum]] = np.nan
    # Accessing the rows without empty values at colnum
    df_complete = df.dropna()
    df_complete.shape
    # Accessing the rows with empty values at colnum
    df_empty = df.iloc[idx]
    df_empty.shape
    # Splitting complete rows into target/features
    features = df_complete.drop(df.columns[[colnum]], axis=1)
    target_variable = df_complete.iloc[:, [colnum]]
    # Splitting the rows with empty colnum into features and response (which is what we're predicting)
    features_empty = df_empty.drop(df.columns[[colnum]], axis=1)
    
    # now can start classifying/ predicting:
    if(flag==0): # classification
        from sklearn.ensemble import RandomForestClassifier
        print('Classifying...')
        print()
        # Training random forest
        randFor = RandomForestClassifier(n_estimators = 20)
        randFor.fit(features, target_variable)
        # Accuracy on the training set set
        print('Training score: ',randFor.score(features, target_variable))
        # Set of "City Group" predictions for the rows with empty values 
        y_pred_randFor = randFor.predict(features_empty)
        print(y_pred_randFor)
    else: # prediction
        from sklearn.linear_model import Ridge
        print('Predicting...')
        print()
        # Ridge Regression
        ridgereg = Ridge(normalize=True)
        ridgereg.fit(features,target_variable)
        y_pred_ridge = ridgereg.predict(features_empty)
        print (y_pred_ridge)

<b> Testing on Restaurant.csv, revenue </b>

In [3]:
original_df = pd.read_csv("Restaurant.csv")
original_df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [4]:
df = original_df.copy()
# List of random numbers (no repeats) between 1 and 137 and then delete A_follower_count[that row]   
rows_to_delete = random.sample(range(137), 30)
# Deleting values from those rows
for x in rows_to_delete:
    df['revenue'][x] = np.nan
# Creating array of the correct answers from original data frame  
deleted_answer_list = []
for x in rows_to_delete:
    deleted_answer_list.append(original_df['revenue'][x])
# Converting it to array from a list so we can perform certain calculations
deleted_answer_array = np.array(deleted_answer_list)
deleted_answer_array

array([  4015749.,   3600467.,   1149870.,  13575224.,   4264176.,
         7217634.,   3752885.,   2344689.,   3807496.,   4250553.,
         2551252.,   3004429.,   8213524.,   4807746.,   2383840.,
         2732645.,   2371202.,   4316715.,   1756069.,   4882985.,
         3836721.,   2525375.,   2954086.,   5787594.,   5653753.,
         2675511.,   4590423.,   9262754.,   4758476.,   3248660.])

In [5]:
# test the function
fillEmpty(df,42,1)

Predicting...

[[ 4705359.66622173]
 [ 2861903.54972608]
 [ 3997522.11962385]
 [ 5208446.77974179]
 [ 5136235.94888857]
 [ 5324599.32264716]
 [ 4143708.80058126]
 [ 4818750.79587835]
 [ 4274971.23788719]
 [ 4454128.13165543]
 [ 4732540.28814412]
 [ 5020942.64806134]
 [ 4672716.03701595]
 [ 4817123.41853048]
 [ 4293130.96551986]
 [ 4728044.1763354 ]
 [ 4408222.97492989]
 [ 3893236.21379395]
 [ 4290612.92517737]
 [ 4117917.77606792]
 [ 5107217.68213713]
 [ 3788116.57220292]
 [ 3984277.75379835]
 [ 2331139.37839418]
 [ 3034647.74021316]
 [ 3144454.85480484]
 [ 4690146.50347888]
 [ 3589572.74695947]
 [ 4716656.10619669]
 [ 4685101.93314578]]


<b> Testing on Restaurant.csv, City Group </b>

In [6]:
df2 = original_df.copy()
# List of random numbers (no repeats) between 1 and 137 and then delete A_follower_count[that row]   
rows_to_delete = random.sample(range(137), 30)
# Deleting values from those rows
for x in rows_to_delete:
    df2['City Group'][x] = np.nan
# Creating array of the correct answers from original data frame  
deleted_answer_list = []
for x in rows_to_delete:
    deleted_answer_list.append(original_df['City Group'][x])
# Converting it to array from a list so we can perform certain calculations
deleted_answer_array_city_group = np.array(deleted_answer_list)
deleted_answer_array_city_group

array(['Big Cities', 'Other', 'Big Cities', 'Big Cities', 'Big Cities',
       'Big Cities', 'Other', 'Other', 'Big Cities', 'Big Cities',
       'Big Cities', 'Other', 'Other', 'Other', 'Big Cities', 'Other',
       'Other', 'Big Cities', 'Big Cities', 'Big Cities', 'Other', 'Other',
       'Other', 'Other', 'Big Cities', 'Other', 'Other', 'Big Cities',
       'Other', 'Other'], 
      dtype='<U10')

In [19]:
# test the function
fillEmpty(df2,3,0)

Classifying...

Training score:  1.0
[ 0.  1.  0.  1.  1.  0.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.
  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.]
